In [1]:
import csv
import numpy as np
import random

def readfile(filename,encode = 'UTF-8'):
    rawlist = []
    with open(filename,'r+', encoding=encode) as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            rawlist.append(row)
    return rawlist

def writefile(filename,inputlist):
    with open(filename,'a+', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for row in inputlist:
            writer.writerow(row)

## 唐跟蘇珊兩邊的碼拌在一起，有把需要的步驟函數化，主要看函數即可

In [2]:
tang = readfile('tang_content.csv')
susan = readfile('susan.csv')

# 做字典

In [3]:
word_dict = {}

for i in range(0,len(tang)):
    for j in range(1,len(tang[i])):
        for char in tang[i][j]:
            if char not in word_dict:
                word_dict[char] = 1
            else:
                word_dict[char] = word_dict[char]+1

for i in range(0,len(susan)):
    for j in range(1,len(susan[i])):
        for char in susan[i][j]:
            if char not in word_dict:
                word_dict[char] = 1
            else:
                word_dict[char] = word_dict[char]+1


# 把字典排序，並拿掉出現頻率太低的字

In [4]:
sorted_word_dict = [(k, word_dict[k]) 
                    for k in sorted(word_dict, key=word_dict.get, reverse=True)]
too_high = 400 
too_low = 20

final_dict = []

for i in range(0,len(sorted_word_dict)):
    if not sorted_word_dict[i][1] < too_low and not sorted_word_dict[i][1] > too_high:
        final_dict.append(sorted_word_dict[i])

In [5]:
len(final_dict)

1038

# 把每一篇文章變成陣列

In [6]:
def normalize(inputlist):
    inputsum = sum(inputlist)
    if not inputsum==0:
        for i in range(0,len(inputlist)):
            if not (inputlist[i]/inputsum) == None:
                inputlist[i] = inputlist[i]/inputsum
            else:
                print('Warning: something weird happened while dividing')
    else:
        print('Warning: input sum==0')
    return inputlist

In [7]:
def articletoVector(article_set,final_dict,cuthead=1):
    variable_bracket = np.zeros(len(final_dict))
    output_vector = []
    tmp = []

    for i in range(0,len(article_set)):
        tmp.append(article_set[i][0])
        for j in range(cuthead,len(article_set[i])):
            for char in article_set[i][j]:
                for pos in range(0,len(final_dict)):
                    if char == final_dict[pos][0]:
                        variable_bracket[pos] = variable_bracket[pos] + 1/final_dict[pos][1]
                        break
            variable_bracket = normalize(variable_bracket)
            tmp.append(variable_bracket)    
            variable_bracket = np.zeros(len(final_dict))
        output_vector.append(tmp)
        tmp = [] 
    return(output_vector)

In [8]:
susan_vector = articletoVector(susan,final_dict)

In [9]:
variable_bracket = np.zeros(len(final_dict))
tang_vector = []
tmp = []

for i in range(0,len(tang)):
    tmp.append(tang[i][0])
    for j in range(1,len(tang[i])):
        #print(len(tang[i][j]))
        for char in tang[i][j]:
            for pos in range(0,len(final_dict)):
                if char == final_dict[pos][0]:
                    variable_bracket[pos] = variable_bracket[pos] + 1/final_dict[pos][1]
                    break
        variable_bracket = normalize(variable_bracket)
        tmp.append(variable_bracket)    
        variable_bracket = np.zeros(len(final_dict))
    tang_vector.append(tmp)
    tmp = []  

In [11]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# 引入每篇文章的分數，並結合tang_vector產生train_source

In [12]:
score = [0,5,5,5,4,4,4,3,3,2,2,1,1]
train_source = []
tmp = []
for i in range(0,len(tang_vector)):
    for pos in range(1,len(tang_vector[i])):
        tmp.append(score[pos])
        for item in tang_vector[i][pos]:
            tmp.append(item)
        train_source.append(tmp)
        tmp = []

In [13]:
def create_train_data(raw_vector,score=[0],with_score=0,cuthead=1):
    output = []
    tmp = []
    for i in range(0,len(raw_vector)):
        for pos in range(cuthead,len(raw_vector[i])):
            if with_score==1:
                tmp.append(score[pos])
            for item in raw_vector[i][pos]:
                tmp.append(item)
            output.append(tmp)
            tmp = []
    return output

# 準備測試資料

In [14]:
X = train_source
random.shuffle(X)
Y = []
for i in range(0,len(X)):
    Y.append(train_source[i][0])
    del X[i][0]

# Split the data into training/testing sets
X_train = X[:2700]
X_test = X[2700:]

# Split the targets into training/testing sets
Y_train = Y[:2700]
Y_test = Y[2700:]

print(len(X_train))
print(len(X_test))

2700
1063


# Logistic Regression 表現次佳

In [15]:
# Create linear regression object
regr = linear_model.LogisticRegression()

# Train the model using the training sets
regr.fit(X_train, Y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
Y_pred = regr.predict(X_test)

In [25]:
correct=0
soso=0

for i in range(0,len(Y_test)):
    if abs(Y_pred[i]-Y_test[i])==0:
        correct = correct+1
    elif abs(Y_pred[i]-Y_test[i])==1:
        soso=soso+1
print('Logistic Regression: %f3'%(correct/len(Y_test)),'%f3'%((soso+correct)/len(Y_test)))

Logistic Regression: 0.3828793 0.7102543


# Support Vector Machine 表現在標準化後不太好

In [27]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(X_train, Y_train)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [28]:
Y_pred = clf.predict(X_test)

In [29]:
correct=0
soso=0

for i in range(0,len(Y_test)):
    if abs(Y_pred[i]-Y_test[i])==0:
        correct = correct+1
    elif abs(Y_pred[i]-Y_test[i])==1:
        soso=soso+1
print('SVM: %f3'%(correct/len(Y_test)),'%f3'%((soso+correct)/len(Y_test)))

SVM: 0.2314213 0.6688623


# Random Forest 表現最佳

In [30]:
from sklearn import ensemble
Rforest = ensemble.RandomForestClassifier(n_estimators=1000, random_state=0)  
Rforest.fit(X_train, Y_train)  

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [32]:
Y_pred = Rforest.predict(X_test)

In [33]:
correct=0
soso=0

for i in range(0,len(Y_test)):
    if abs(Y_pred[i]-Y_test[i])==0:
        correct = correct+1
    elif abs(Y_pred[i]-Y_test[i])==1:
        soso=soso+1
print('Random Forest: %f3'%(correct/len(Y_test)),'%f3'%((soso+correct)/len(Y_test)))

Random Forest: 0.4261523 0.7676393


# 用唐的模型拿來算蘇珊的分數，很怪的方法，下次不敢了

In [445]:
X_susan = create_train_data(susan_vector)
susan_predict = classifier.predict(X_susan)

In [488]:
susan_predict_output = []
tmp = []
tot = 0
for i in range(len(susan)):
    tmp.append(susan[i][0])
    for j in range(0,12):
        tmp.append(susan_predict[tot+j])
    tot = tot+12
    susan_predict_output.append(tmp)
    tmp = []
writefile('susan_daily_score.csv',susan_predict_output)

# 蘇珊周運勢的計算

In [513]:
susan_week = readfile('susan_week.csv')
susan_week_raw = []
for i in range(0,len(susan_week)):
    susan_week_raw.append([susan_week[i][0]])
    
susan_week_vector = articletoVector(susan_week_raw,final_dict,cuthead=0)
susan_week_X = create_train_data(susan_week_vector)

In [516]:
Y_pred = classifier.predict(susan_week_X)
print(Y_pred)

[4 4 4 4 3 4 4 3 5 2 1 4 5 1 2 4 4 2 4 5 4 3 3 5 4 1 3 5 2 4 1 1 4 1 4 4 3
 1 3 1 4 4 1 3 4 5 5 1 4 4 1 1 5 4 5 4 4 4 2 4 2 5 2 2 1 4 5 2 5 4 1 1 4 1
 3 4 4 4 1 1 4 5 4 4 4 1 1 2 5 4 5 1 4 1 1 5 4 4 1 1 5 1 4 2 4 1 3 3 3 4 1
 5 3 4 4 1 1 5 4 3 4 1 5 1 5 4 5 5 3 3 1 3 4 5 2 1 4 4 1 1 3 5 1 1 4 1 4 1
 4 1 1 1 4 2 4 1]


In [517]:
for i in range(0,len(susan_week)):
    susan_week[i].append(Y_pred[i])
writefile('susan_week_with_value.csv',susan_week)

# 唐周運勢的計算，因為文字量太少，有算跟沒算一樣

In [519]:
tang_week = readfile('tang_week.csv')
tang_week_raw = []
for i in range(0,len(tang_week)):
    tang_week_raw.append([tang_week[i][0]])

In [525]:
tang_week_vector = articletoVector(tang_week_raw,final_dict,cuthead=0)
tang_week_X = create_train_data(tang_week_vector)
Y_pred = classifier.predict(tang_week_X)
print(Y_pred)

[1 5 1 5 1 3 5 5 1 1 4 2 1 4 2 1 1 1 1 1 4 4 5 1 1 1 3 4 5 4 1 2 5 5 4 4 2
 1 5 1 1 5 5 5 2 5 4 4 1 1 2 4 5 1 4 1 1 4 5 4 4 1 4 3 4 1 3 1 5 1 1 4 1 4
 4 1 4 5 5 1 5 1 2 5 1 1 4 4 5 2 5 1 2 1 4 3 1 1 1 1 5 2 1 1 4 5 4 1 4 1 2
 5 1 3 1 4 4 4 4 1 4 1 3 4 1 5 1 4 4 5 4 5 2 1 1 4 5 1 1 5 5 5 5 1 4 1 5 1
 2 1 5 5 5 1 4 1]


In [526]:
for i in range(0,len(tang_week)):
    tang_week[i].append(Y_pred[i])
writefile('tang_week_with_value.csv',tang_week)